
# <font color="green"> Problem 1 :  算術式とimportの練習</font>
* https://docs.python.org/ja/3.8/library/math.html を見ながら, 以下を計算する式をPythonで書いて, 実際に答えを出してみよ


* 100の階乗 100! ($= 100 \cdot 99 \cdot \cdots \cdot 2 \cdot 1$) は10進数で何桁か?

In [ ]:
math.floor(math.log10(math.factorial(100))) + 1


#  コメント
* 落とし穴1 : `math.log` は $e$ を底とした対数. 10 を底とした対数は, `math.log(x, 10)` とするか, `math.log10(x)`
* 細かい落とし穴2: logで発生した小数点をどう処理すれば正しい桁数になるのか?

* $x$が$n$桁の整数なのは, $10^{n-1} \leq x < 10^n$ のとき.
$$ \begin{array}{rcl} 10^{n-1} \leq x < 10^n & \iff & n - 1 \leq \log_{10} x < n \end{array} $$
* なので一見すると小数点以下を切り上げれば($\lceil r \rceil$) ただしく$n$が求まる気がするが, 注意してみると, $\log_{10} x$がぴったり整数になるときは正しくない
* 正しい処理は, 切り捨てた後 1 を足す ($\lfloor x \rfloor + 1$) ことである


* 3辺の長さが5, 6, 7の三角形で, 長さ5と6の辺の間の角度は約何度か?

In [ ]:
math.degrees(math.acos((5*5 + 6*6 - 7*7)/(2*5*6)))

* ${}_{2021}\mbox{C}_{37}$を4で割ったあまりはいくらか?

In [ ]:
math.comb(2021, 37) % 4


#  コメント
* ${}_{n}\mbox{C}_{r}$ そのものを計算する関数(comb)が備わっているのでそれを使って, あまりは % で求めてもらえばよいというつもりだったが, 他にもやり方があるので結果として割と落とし穴の多い問題になった
* どちらの落とし穴も本質的には同じ理由で生ずるもので, それはそれで教育的なので解説する



##  別法1: あまりをもとめるのに `math.remainder` 関数を使う

In [ ]:
math.remainder(math.comb(2021, 37), 4)


* すると答えは 0.0 と出る
* これは2つの理由で起きる 
  * `math.remainder(x, y)` が, 計算をする際にx, yを小数点数つきの数, 正確には「浮動小数点数」に変換してから計算する
  * sinやlogなども同様のことをする
  * 「浮動小数点数」は表現できる桁数に限界があり, 巨大な整数を正確に表すことができない (下の何桁かが切り捨てられてしまう)
  * コンピュータには浮動小数点数を高速に扱うハードウェアが備わっているため計算にはしばしば浮動小数点数が使われる(そもそも無理数など, 有限桁数ではどうやっても正確に表しようのない数があるので適度なところで妥協しなくてはならない)
* 今の場合, x が巨大なので上記のような限界に引っかかってしまう

* 「浮動小数点数」が数をどのように表現しているかを(多少端折って)説明すると, 

$$\pm 1.ddd \cdots ddd \times 2^{nnn \cdots nnn}$$

という形ですべての数を表現している. ここで,
 * $ddd ... ddd$ の部分は2進数で52 桁,
 * $nnn ... nnn$ は2進数で11桁 (うち1桁は正負の符号を表す)
と決まっている. 
* 負の数をどうするとか, 0をどうするかとか細かい話はあるが省略(<a href="https://en.wikipedia.org/wiki/Double-precision_floating-point_format" target="_blank">詳細</a>).

* したがって大きな整数は, 1ddd ... ddd (合計53桁) までの整数しか正確に表せない
* つまり正確に表せる整数の最大値は, 
$$ 1.111 \cdots 111 \times 2^{53} = 2^{54} - 1$$ 
までということになる
* それを超えると, 下の方の桁が0になったのと同じになる

* そのことは以下のようにして確かめられる
* float(x) は x を浮動小数点数に変換する関数

* $2^{54} - 2 = 2^{54} - 1$ か?


In [ ]:
float(2 ** 54 - 2) == float(2 ** 54 - 1)

* $2^{54} - 1 = 2^{54}$ か?

In [ ]:
float(2 ** 54 - 1) == float(2 ** 54)

In [ ]:
float(2 ** 54 - 0) - float(2 ** 54 - 1)


* floatを使わなければこのようなことはおきない


In [ ]:
(2 ** 54) - (2 ** 54 - 1)


* なおこれはコンピュータが, 整数ならいくらでも桁数の多い値を表せるハードウェアを備えているということではなく, Pythonが桁数の大きい整数になったら複数の(桁数の限られた)整数をつなぎ合わせて(例えば 64 桁の整数を2つあわせれば 128 桁分の整数が表せる)大きな数を表しているからできることである
* 他の言語の整数ではこうはならないことが普通なので注意が必要



##  別法2: combを使わずに, factorial とわり算を使う
$${}_{n}\mbox{C}_{r} = \frac{n!}{r! (n-r)!}$$

だから, `math.factorial`を使って以下のようにする


In [ ]:
(math.factorial(2021) / (math.factorial(37) * math.factorial(2021 - 37))) % 4


* こちらも答えは0.0 と出てしまう
* 理由もほとんど同じで `/` が「答えを浮動小数点数で求める」と決まっているため
* すなわち上記は以下と同じこと. そしてそれが正しく答えを求められない理由も別法1と全く同じ


In [ ]:
float(math.factorial(2021) / (math.factorial(37) * math.factorial(2021 - 37))) % 4


* この方針で正しく答えを求めるには `/` を `//` に置き換える
* `//` は「答えを整数で求める割り算」(端数は切り捨てる)


In [ ]:
math.factorial(2021) // (math.factorial(37) * math.factorial(2021 - 37)) % 4